In [2]:
import os
os.chdir(os.path.dirname(os.path.realpath(__file__)))
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt


def load_SMHI_data(file_name, path):
    for i in range(6, 12):
        try:
            df = pd.read_csv(path+file_name, header=i, sep=";")
        except:
            continue
        if df.columns[0] != "Datum":
            continue
        else:
            print(file_name, df.shape)
            return df
    raise OverflowError("Number of iterations exceeded")
    
def retrieve_datetime(df: pd.DataFrame) -> pd.Series:
    return pd.to_datetime(df["Datum"] + " " + df["Tid (UTC)"])
    
# Läs in datan från SMHI
dataframes = [load_SMHI_data(file, "data_24-11-29/") for file in os.listdir("data_24-11-29")]

# Läs in radon-datan
df_main = pd.read_csv("radon_2024-11-30.csv", sep=",", header=5)

# Formatera 
df_main.columns = ["Datum", "Radon"]
df_main["Datum"] = pd.to_datetime(df_main["Datum"], format = "%d/%m/%Y %H:%M").dt.round("h")

parameters = ['Lufttemperatur',  'Vindriktning', 'Vindhastighet', 'Rådande väder', 
              'Lufttryck reducerat havsytans nivå',  'Nederbördsmängd', 'Global Irradians (svenska stationer)',
              'Solskenstid']

df_main.index = pd.DatetimeIndex(pd.to_datetime(df_main["Datum"]))


idx = pd.date_range(df_main.index[0], df_main.index[-1], freq="h")

for df in dataframes:
    df.index = pd.DatetimeIndex(retrieve_datetime(df))
    df = df.reindex(idx, fill_value=0)
    for par in parameters:
        if par in df.columns:
            df_main.insert(df_main.shape[1], par, df[par])

# for par in parameters:
#     df_main.plot(y="Radon", x=par, kind="scatter")

df_main.describe()

df_main = df_main.iloc[:,1:]

df_train = df_main.dropna().sample(frac=0.8, random_state=0)
df_test = df_main.dropna().drop(df_train.index)

train_labels = df_train.pop("Radon")
test_labels = df_test.pop("Radon")

NameError: name '__file__' is not defined

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

normalizer = tf.keras.layers.Normalization(axis=-1)

normalizer.adapt(np.array(df_train))
print(normalizer.mean.numpy())

def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01))
  return model


dnn_model = build_and_compile_model(normalizer)

%%time
history = dnn_model.fit(
    df_train,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=300)

2024-12-13 15:35:03.626841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'np' is not defined

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='validation_loss')
  plt.ylim([0, 100])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)
  
plot_loss(history)

In [ ]:
dnn_model.evaluate(df_test, test_labels, verbose=0)

x = tf.linspace(1, 20, 420)
y = dnn_model.predict(df_test)
test_result = dnn_model.evaluate(df_test, test_labels)
print(test_result)

plt.plot(x, y)
plt.plot(x, test_labels)
plt.ylabel("Radon")